In [1]:
import numpy as np 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils.np_utils import to_categorical
import re

In [3]:
data = pd.read_csv('C:\\Sentiment.csv')
data = data[['text','sentiment']]
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

4472
16986


In [4]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 28, 128)           256000    
                                                                 
 spatial_dropout1d (Spatial  (None, 28, 128)           0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 511194 (1.95 MB)
Trainable params: 511194 (1.95 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [5]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7188, 28) (7188, 2)
(3541, 28) (3541, 2)


In [6]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
225/225 - 121s - loss: 0.4362 - accuracy: 0.8132 - 121s/epoch - 540ms/step
Epoch 2/7
225/225 - 120s - loss: 0.3173 - accuracy: 0.8653 - 120s/epoch - 534ms/step
Epoch 3/7
225/225 - 120s - loss: 0.2773 - accuracy: 0.8855 - 120s/epoch - 534ms/step
Epoch 4/7
225/225 - 117s - loss: 0.2505 - accuracy: 0.8976 - 117s/epoch - 522ms/step
Epoch 5/7
225/225 - 130s - loss: 0.2216 - accuracy: 0.9097 - 130s/epoch - 579ms/step
Epoch 6/7
225/225 - 127s - loss: 0.1999 - accuracy: 0.9220 - 127s/epoch - 565ms/step
Epoch 7/7
225/225 - 123s - loss: 0.1802 - accuracy: 0.9252 - 123s/epoch - 547ms/step


In [8]:
validation_size = 1500
X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

17/17 - 1s - loss: 0.5574 - accuracy: 0.8041 - 1s/epoch - 62ms/step
score: 0.56
acc: 0.80


In [9]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1
print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")


1/1 - 1s - 1s/epoch - 1s/step
1/1 - 0s - 63ms/epoch - 63ms/step
1/1 - 0s - 63ms/epoch - 63ms/step
1/1 - 0s - 63ms/epoch - 63ms/step
1/1 - 0s - 72ms/epoch - 72ms/step
1/1 - 0s - 67ms/epoch - 67ms/step
1/1 - 0s - 63ms/epoch - 63ms/step
1/1 - 0s - 91ms/epoch - 91ms/step
1/1 - 0s - 47ms/epoch - 47ms/step
1/1 - 0s - 63ms/epoch - 63ms/step
1/1 - 0s - 68ms/epoch - 68ms/step
1/1 - 0s - 78ms/epoch - 78ms/step
1/1 - 0s - 63ms/epoch - 63ms/step
1/1 - 0s - 84ms/epoch - 84ms/step
1/1 - 0s - 78ms/epoch - 78ms/step
1/1 - 0s - 63ms/epoch - 63ms/step
1/1 - 0s - 63ms/epoch - 63ms/step
1/1 - 0s - 63ms/epoch - 63ms/step
1/1 - 0s - 47ms/epoch - 47ms/step
1/1 - 0s - 196ms/epoch - 196ms/step
1/1 - 0s - 63ms/epoch - 63ms/step
1/1 - 0s - 63ms/epoch - 63ms/step
1/1 - 0s - 71ms/epoch - 71ms/step
1/1 - 0s - 63ms/epoch - 63ms/step
1/1 - 0s - 62ms/epoch - 62ms/step
1/1 - 0s - 63ms/epoch - 63ms/step
1/1 - 0s - 63ms/epoch - 63ms/step
1/1 - 0s - 94ms/epoch - 94ms/step
1/1 - 0s - 78ms/epoch - 78ms/step
1/1 - 0s - 63ms/

In [10]:
twt = ['Meetings: Because none of us is as dumb as all of us.']
twt = tokenizer.texts_to_sequences(twt)
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")


[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0  206  633    6  150    5   55 1055   55   46    6  150]]
1/1 - 0s - 112ms/epoch - 112ms/step
negative
